# Reprocessing procedure for Las Campanas Observatory Swope data using BANZAI-Imaging
## Matt Daily

## Set up relevant runtime configuration

In [1]:
import logging
from glob import glob
import os
import re

from banzai.calibrations import make_master_calibrations
from banzai import settings
from banzai import dbs
from banzai.utils.stage_utils import run_pipeline_stages
from banzai.logs import set_log_level
import banzai.main

import pkg_resources
import requests
from astropy.io import fits


set_log_level('DEBUG')
logger = logging.getLogger('banzai')

os.environ['OPENTSDB_PYTHON_METRICS_TEST_MODE'] = 'True'
os.environ['DB_ADDRESS'] = 'sqlite:///test.db'

settings.processed_path= os.path.join(os.getcwd(), 'test_data')
settings.fpack=True
settings.db_address = os.environ['DB_ADDRESS']
settings.no_bpm = True
settings.reduction_level = 91

# set up the context object.
context = banzai.main.parse_args(settings, parse_system_args=False)

ERROR:root:ConnectionError(MaxRetryError("HTTPConnectionPool(host='scheduler-dev.lco.gtn', port=80): Max retries exceeded with url: /api/version (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x127591550>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))"))


### Take the data from the directory and combine it into MEFs
This branch of BANZAI expects that each amplifier have an extension in an MEF. BANZAI can then do the work of stitching them together.

In [2]:
raw_data_dir = os.path.join(os.getcwd(), 'swope_data', 'raw')
mef_combined_data_dir = os.path.join(os.getcwd(), 'swope_data', 'mef_combined')
processed_data_dir = os.path.join(os.getcwd(), 'swope_data', 'processed')

In [3]:
prefix_set = set()
for file_path in glob(os.path.join(raw_data_dir, 'ccd1144*.fits.fz')):
    # extract the unique prefixes from the raw data directory
    prefix_set.add(re.sub('c\d', '', file_path))

In [4]:
# now create MEFs from the groupings!
for prefix in prefix_set:
    filepaths = [prefix.replace('.fits.fz', f'c{i}.fits.fz') for i in [4,2,3,1]]

    image_hdus = fits.HDUList([fits.PrimaryHDU()])
    for path in filepaths:
        image_hdus.append(fits.open(path)['COMPRESSED_IMAGE'])

    # Now do some munging of the headers.
    # 1 is top right, rotate 90 counterclockwise and invert x (x -> -y, y-> -x)
    image_hdus[1].data = image_hdus[1].data.T[::-1, ::-1]
    image_hdus[1].header['DATASEC'] = '[129:2184,129:2176]'
    image_hdus[1].header['BIASSEC'] = '[1:128,1:2176]'
    image_hdus[1].header['DETSEC'] = '[2057:4112,2049:4096]'
    image_hdus[1].header.pop('TRIMSEC')

    # 2 is bottom left rotate 90 clockwise and invert x (y -> x x -> y)
    image_hdus[2].data = image_hdus[2].data.T
    image_hdus[2].header['DATASEC'] = '[1:2056,1:2048]'
    image_hdus[2].header['BIASSEC'] ='[2057:2084,1:2176]'
    image_hdus[2].header['DETSEC'] = '[1:2056,1:2048]'
    image_hdus[2].header.pop('TRIMSEC')

    # 3 bottom right rotate 90 counterclockwise (x-> -y, y - > x)
    image_hdus[3].data = image_hdus[3].data.T[:, ::-1]
    image_hdus[3].header['DATASEC'] = '[129:2184,1:2048]'
    image_hdus[3].header['BIASSEC'] ='[1:129,1:2176]'
    image_hdus[3].header['DETSEC'] = '[2057:4112,1:2048]'
    image_hdus[3].header.pop('TRIMSEC')

    # 4 is top left 270 deg  counterclockwise (x -> y, y->-x)
    image_hdus[4].data = image_hdus[4].data.T[::-1, :]
    image_hdus[4].header['DATASEC'] = '[1:2056,129:2176]'
    image_hdus[4].header['BIASSEC'] ='[2057:2184,1:2176]'
    image_hdus[4].header['DETSEC'] = '[1:2056, 2049:4096]'
    image_hdus[4].header.pop('TRIMSEC')

    # Copy all overlapping values into the main header
    for i in image_hdus[1].header:
        if i == 'COMMENT' or len(i) == 0:
            continue
        if image_hdus[2].header.get(i) == image_hdus[1].header.get(i):
            image_hdus[0].header[i] = image_hdus[1].header[i]

    image_hdus[0].header['TRIMSEC'] = '[1:4112,1:4096]'

    for i in range(1,5):
        image_hdus[i].header['RDNOISE'] = image_hdus[i].header['ENOISE']
        image_hdus[i].header['GAIN']  = image_hdus[i].header['EGAIN'] 

    # now create an HDUList and a unique filename to store the data in
    filename_suffix_obstype_mapping = {'Bias': 'b00',
                                       'Flat': 'f00',
                                       'Object': 'e00'}
    hdu_list = fits.HDUList(image_hdus)
    filename = f'{image_hdus[1].header["FILENAME"][:-2]}-{image_hdus[1].header["UT-DATE"].replace("-", "")}-{filename_suffix_obstype_mapping[image_hdus[1].header["EXPTYPE"]]}.fits.fz'
    hdu_list.writeto(os.path.join(mef_combined_data_dir, filename), overwrite=True)

In [5]:
if os.path.exists(os.path.join(os.getcwd(), 'test.db')):
    os.remove(os.path.join(os.getcwd(), 'test.db'))
if os.path.exists(context.processed_path):
    os.system(f'rm -r {context.processed_path}')
os.system(f'banzai_create_db --db-address={os.environ["DB_ADDRESS"]}')
os.system(f'banzai_add_site --site LCO --latitude -29.08300 --longitude -70.698005 --elevation 2280 --timezone -4 --db-address={os.environ["DB_ADDRESS"]}')
os.system(f'banzai_add_instrument --site LCO --camera Direct/4Kx4K-4 --name Direct/4Kx4K-4 --instrument-type swope-imager --db-address={os.environ["DB_ADDRESS"]}')

0

In [6]:
instrument = dbs.get_instruments_at_site('LCO', settings.db_address)[0]

In [7]:
bias_files = glob(os.path.join(mef_combined_data_dir, '*b00*'))
for bias_file in bias_files:
    run_pipeline_stages([{'path': bias_file}], context)

2023-03-23 10:55:45.117     INFO:          stages: Running banzai.stages.BadPixelMaskLoader | {"filename": "ccd1533-20220925-b00.fits.fz", "site": "LCO", "instrument": "Direct/4Kx4K-4", "epoch": "20220925", "request_num": null, "obstype": "BIAS", "filter": "r", "processName": "MainProcess"}


INFO:banzai:Running banzai.stages.BadPixelMaskLoader | {"filename": "ccd1533-20220925-b00.fits.fz", "site": "LCO", "instrument": "Direct/4Kx4K-4", "epoch": "20220925", "request_num": null, "obstype": "BIAS", "filter": "r", "processName": "MainProcess"}


2023-03-23 10:55:45.128    ERROR:             bpm: Master BPM does not exist | {"filename": "ccd1533-20220925-b00.fits.fz", "site": "LCO", "instrument": "Direct/4Kx4K-4", "epoch": "20220925", "request_num": null, "obstype": "BIAS", "filter": "r", "processName": "MainProcess"}


ERROR:banzai:Master BPM does not exist | {"filename": "ccd1533-20220925-b00.fits.fz", "site": "LCO", "instrument": "Direct/4Kx4K-4", "epoch": "20220925", "request_num": null, "obstype": "BIAS", "filter": "r", "processName": "MainProcess"}


2023-03-23 10:55:45.297     INFO:          stages: Running banzai.stages.OverscanSubtractor | {"filename": "ccd1533-20220925-b00.fits.fz", "site": "LCO", "instrument": "Direct/4Kx4K-4", "epoch": "20220925", "request_num": null, "obstype": "BIAS", "filter": "r", "processName": "MainProcess"}


...

INFO:banzai:Running banzai.stages.BiasMasterLevelSubtractor | {"filename": "ccd1533-20220925-b00.fits.fz", "site": "LCO", "instrument": "Direct/4Kx4K-4", "epoch": "20220925", "request_num": null, "obstype": "BIAS", "filter": "r", "processName": "MainProcess"}


2023-03-23 10:55:48.311     INFO:          stages: Running banzai.stages.BiasComparer | {"filename": "ccd1533-20220925-b00.fits.fz", "site": "LCO", "instrument": "Direct/4Kx4K-4", "epoch": "20220925", "request_num": null, "obstype": "BIAS", "filter": "r", "processName": "MainProcess"}


INFO:banzai:Running banzai.stages.BiasComparer | {"filename": "ccd1533-20220925-b00.fits.fz", "site": "LCO", "instrument": "Direct/4Kx4K-4", "epoch": "20220925", "request_num": null, "obstype": "BIAS", "filter": "r", "processName": "MainProcess"}


2023-03-23 10:55:48.320    ERROR:    calibrations: No master bias to compare to, Flagging image as bad. | {"filename": "ccd1533-20220925-b00.fits.fz", "site": "LCO", "instrument": "Direct/4Kx4K-4", "epoch": "20220925", "request_num": null, "obstype": "BIAS", "filter": "r", "processName": "MainProcess"}


ERROR:banzai:No master bias to compare to, Flagging image as bad. | {"filename": "ccd1533-20220925-b00.fits.fz", "site": "LCO", "instrument": "Direct/4Kx4K-4", "epoch": "20220925", "request_num": null, "obstype": "BIAS", "filter": "r", "processName": "MainProcess"}


In [8]:
def mark_frames_as_good(filenames):
    for filename in glob(f'test_data/LCO/*/*/processed/{filenames}'):
        dbs.mark_frame(os.path.basename(filename), "good", db_address=os.environ['DB_ADDRESS'])

In [9]:
mark_frames_as_good('*b91*')

In [10]:
make_master_calibrations(instrument, 'Bias', '2022-09-20', '2022-09-30', context)

2023-03-23 10:55:51.504     INFO:    calibrations: Making master frames | {"type": "swope-imager", "site": "LCO", "camera": "Direct/4Kx4K-4", "obstype": "Bias", "min_date": "2022-09-20", "max_date": "2022-09-30", "processName": "MainProcess"}


INFO:banzai:Making master frames | {"type": "swope-imager", "site": "LCO", "camera": "Direct/4Kx4K-4", "obstype": "Bias", "min_date": "2022-09-20", "max_date": "2022-09-30", "processName": "MainProcess"}


2023-03-23 10:55:52.193     INFO:          stages: Running banzai.stages.BiasMaker | {"filename": "ccd1533-20220925-b91.fits.fz", "site": "LCO", "instrument": "Direct/4Kx4K-4", "epoch": "20220925", "request_num": null, "obstype": "BIAS", "filter": "r", "processName": "MainProcess"}


INFO:banzai:Running banzai.stages.BiasMaker | {"filename": "ccd1533-20220925-b91.fits.fz", "site": "LCO", "instrument": "Direct/4Kx4K-4", "epoch": "20220925", "request_num": null, "obstype": "BIAS", "filter": "r", "processName": "MainProcess"}


2023-03-23 10:55:54.322     INFO:    calibrations: Created master calibration stack | {"filename": "4Kx4K-4-20220925-bias-bin1x1.fits", "site": "LCO", "instrument": "Direct/4Kx4K-4", "epoch": "20220925", "request_num": null, "obstype": "BIAS", "filter": "r", "calibration_type": "BIAS", "processName": "MainProcess"}


INFO:banzai:Created master calibration stack | {"filename": "4Kx4K-4-20220925-bias-bin1x1.fits", "site": "LCO", "instrument": "Direct/4Kx4K-4", "epoch": "20220925", "request_num": null, "obstype": "BIAS", "filter": "r", "calibration_type": "BIAS", "processName": "MainProcess"}


2023-03-23 10:55:58.364     INFO:    calibrations: Finished | {"processName": "MainProcess"}


INFO:banzai:Finished | {"processName": "MainProcess"}


In [11]:
flat_files = glob(os.path.join(mef_combined_data_dir, '*f00*'))
for flat_file in flat_files:
    run_pipeline_stages([{'path': flat_file}], context)

2023-03-23 10:55:59.393     INFO:          stages: Running banzai.stages.BadPixelMaskLoader | {"filename": "ccd1083-20220925-f00.fits.fz", "site": "LCO", "instrument": "Direct/4Kx4K-4", "epoch": "20220925", "request_num": null, "obstype": "FLAT", "filter": "r", "processName": "MainProcess"}


INFO:banzai:Running banzai.stages.BadPixelMaskLoader | {"filename": "ccd1083-20220925-f00.fits.fz", "site": "LCO", "instrument": "Direct/4Kx4K-4", "epoch": "20220925", "request_num": null, "obstype": "FLAT", "filter": "r", "processName": "MainProcess"}


2023-03-23 10:55:59.403    ERROR:             bpm: Master BPM does not exist | {"filename": "ccd1083-20220925-f00.fits.fz", "site": "LCO", "instrument": "Direct/4Kx4K-4", "epoch": "20220925", "request_num": null, "obstype": "FLAT", "filter": "r", "processName": "MainProcess"}


ERROR:banzai:Master BPM does not exist | {"filename": "ccd1083-20220925-f00.fits.fz", "site": "LCO", "instrument": "Direct/4Kx4K-4", "epoch": "20220925", "request_num": null, "obstype": "FLAT", "filter": "r", "processName": "MainProcess"}


2023-03-23 10:55:59.590     INFO:          stages: Running banzai.stages.OverscanSubtractor | {"filename": "ccd1083-20220925-f00.fits.fz", "site": "LCO", "instrument": "Direct/4Kx4K-4", "epoch": "20220925", "request_num": null, "obstype": "FLAT", "filter": "r", "processName": "MainProcess"}


...

INFO:banzai:No pattern noise found. | {"filename": "ccd1083-20220925-f00.fits.fz", "site": "LCO", "instrument": "Direct/4Kx4K-4", "epoch": "20220925", "request_num": null, "obstype": "FLAT", "filter": "r", "snr_threshold": 10.0, "min_fraction_pixels_above_threshold": 0.01, "min_adjacent_pixels": 3, "fraction_pixels_above_threshold": 0.0, "processName": "MainProcess"}


2023-03-23 10:56:04.583     INFO:          stages: Running banzai.stages.FlatComparer | {"filename": "ccd1083-20220925-f00.fits.fz", "site": "LCO", "instrument": "Direct/4Kx4K-4", "epoch": "20220925", "request_num": null, "obstype": "FLAT", "filter": "r", "processName": "MainProcess"}


INFO:banzai:Running banzai.stages.FlatComparer | {"filename": "ccd1083-20220925-f00.fits.fz", "site": "LCO", "instrument": "Direct/4Kx4K-4", "epoch": "20220925", "request_num": null, "obstype": "FLAT", "filter": "r", "processName": "MainProcess"}


2023-03-23 10:56:04.593    ERROR:    calibrations: No master FLAT to compare to, Flagging image as bad. | {"filename": "ccd1083-20220925-f00.fits.fz", "site": "LCO", "instrument": "Direct/4Kx4K-4", "epoch": "20220925", "request_num": null, "obstype": "FLAT", "filter": "r", "processName": "MainProcess"}


ERROR:banzai:No master FLAT to compare to, Flagging image as bad. | {"filename": "ccd1083-20220925-f00.fits.fz", "site": "LCO", "instrument": "Direct/4Kx4K-4", "epoch": "20220925", "request_num": null, "obstype": "FLAT", "filter": "r", "processName": "MainProcess"}


In [12]:
mark_frames_as_good('*f91*')

In [13]:
make_master_calibrations(instrument, 'Flat', '2022-09-20', '2022-09-30', context)

2023-03-23 10:56:08.488     INFO:    calibrations: Making master frames | {"type": "swope-imager", "site": "LCO", "camera": "Direct/4Kx4K-4", "obstype": "Flat", "min_date": "2022-09-20", "max_date": "2022-09-30", "processName": "MainProcess"}


INFO:banzai:Making master frames | {"type": "swope-imager", "site": "LCO", "camera": "Direct/4Kx4K-4", "obstype": "Flat", "min_date": "2022-09-20", "max_date": "2022-09-30", "processName": "MainProcess"}


2023-03-23 10:56:09.192     INFO:          stages: Running banzai.stages.FlatMaker | {"filename": "ccd1083-20220925-f91.fits.fz", "site": "LCO", "instrument": "Direct/4Kx4K-4", "epoch": "20220925", "request_num": null, "obstype": "FLAT", "filter": "r", "processName": "MainProcess"}


INFO:banzai:Running banzai.stages.FlatMaker | {"filename": "ccd1083-20220925-f91.fits.fz", "site": "LCO", "instrument": "Direct/4Kx4K-4", "epoch": "20220925", "request_num": null, "obstype": "FLAT", "filter": "r", "processName": "MainProcess"}


2023-03-23 10:56:11.334     INFO:    calibrations: Created master calibration stack | {"filename": "4Kx4K-4-20220925-flat-bin1x1-r.fits", "site": "LCO", "instrument": "Direct/4Kx4K-4", "epoch": "20220925", "request_num": null, "obstype": "FLAT", "filter": "r", "calibration_type": "FLAT", "processName": "MainProcess"}


INFO:banzai:Created master calibration stack | {"filename": "4Kx4K-4-20220925-flat-bin1x1-r.fits", "site": "LCO", "instrument": "Direct/4Kx4K-4", "epoch": "20220925", "request_num": null, "obstype": "FLAT", "filter": "r", "calibration_type": "FLAT", "processName": "MainProcess"}


2023-03-23 10:56:15.363     INFO:    calibrations: Finished | {"processName": "MainProcess"}


INFO:banzai:Finished | {"processName": "MainProcess"}


In [14]:
science_files = glob(os.path.join(mef_combined_data_dir, '*e00*'))
for science_file in science_files:
    run_pipeline_stages([{'path': science_file}], context)



2023-03-23 10:56:16.390     INFO:          stages: Running banzai.stages.BadPixelMaskLoader | {"filename": "ccd1088-20220925-e00.fits.fz", "site": "LCO", "instrument": "Direct/4Kx4K-4", "epoch": "20220925", "request_num": null, "obstype": "OBJECT", "filter": "r", "processName": "MainProcess"}


INFO:banzai:Running banzai.stages.BadPixelMaskLoader | {"filename": "ccd1088-20220925-e00.fits.fz", "site": "LCO", "instrument": "Direct/4Kx4K-4", "epoch": "20220925", "request_num": null, "obstype": "OBJECT", "filter": "r", "processName": "MainProcess"}


2023-03-23 10:56:16.401    ERROR:             bpm: Master BPM does not exist | {"filename": "ccd1088-20220925-e00.fits.fz", "site": "LCO", "instrument": "Direct/4Kx4K-4", "epoch": "20220925", "request_num": null, "obstype": "OBJECT", "filter": "r", "processName": "MainProcess"}


ERROR:banzai:Master BPM does not exist | {"filename": "ccd1088-20220925-e00.fits.fz", "site": "LCO", "instrument": "Direct/4Kx4K-4", "epoch": "20220925", "request_num": null, "obstype": "OBJECT", "filter": "r", "processName": "MainProcess"}


2023-03-23 10:56:16.570     INFO:          stages: Running banzai.stages.OverscanSubtractor | {"filename": "ccd1088-20220925-e00.fits.fz", "site": "LCO", "instrument": "Direct/4Kx4K-4", "epoch": "20220925", "request_num": null, "obstype": "OBJECT", "filter": "r", "processName": "MainProcess"}


...

INFO:banzai:Running banzai.stages.WCSSolver | {"filename": "ccd1144-20220925-e00.fits.fz", "site": "LCO", "instrument": "Direct/4Kx4K-4", "epoch": "20220925", "request_num": null, "obstype": "OBJECT", "filter": "r", "processName": "MainProcess"}
  warnings.warn(

  warnings.warn(



2023-03-23 10:59:11.949     INFO:      astrometry: Attempted WCS Solve | {"filename": "ccd1144-20220925-e00.fits.fz", "site": "LCO", "instrument": "Direct/4Kx4K-4", "epoch": "20220925", "request_num": null, "obstype": "OBJECT", "filter": "r", "WCSERR": 0, "processName": "MainProcess"}


INFO:banzai:Attempted WCS Solve | {"filename": "ccd1144-20220925-e00.fits.fz", "site": "LCO", "instrument": "Direct/4Kx4K-4", "epoch": "20220925", "request_num": null, "obstype": "OBJECT", "filter": "r", "WCSERR": 0, "processName": "MainProcess"}


2023-03-23 10:59:11.951     INFO:          stages: Running banzai.stages.PhotometricCalibrator | {"filename": "ccd1144-20220925-e00.fits.fz", "site": "LCO", "instrument": "Direct/4Kx4K-4", "epoch": "20220925", "request_num": null, "obstype": "OBJECT", "filter": "r", "processName": "MainProcess"}


INFO:banzai:Running banzai.stages.PhotometricCalibrator | {"filename": "ccd1144-20220925-e00.fits.fz", "site": "LCO", "instrument": "Direct/4Kx4K-4", "epoch": "20220925", "request_num": null, "obstype": "OBJECT", "filter": "r", "processName": "MainProcess"}
